# Сравнение стратегий распределённого обучения

В этом ноутбуке мы сравним производительность трёх стратегий обучения:

1. **Single GPU (Baseline)** - обучение на одной GPU
2. **DistributedDataParallel (DDP)** - эффективная распределённая параллелизация

Для каждой конфигурации мы запустим профилировщик и выведем результаты.

**Параметры:** 10 эпох обучения для каждого эксперимента.

**Примечание:** Для DDP требуется несколько GPU.


In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.profilers import SimpleProfiler
import time

from text_features_data import TextFeaturesDataModule
from mlp_model import DeepMLP
from lightning_module import TextClassificationModule

# Проверяем наличие GPU (без инициализации CUDA контекста)
# ВАЖНО: не вызываем torch.cuda.get_device_name() и get_device_properties()
# чтобы не инициализировать CUDA до создания DDP процессов
num_gpus = torch.cuda.device_count() if torch.cuda.is_available() else 0

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs available: {num_gpus}")

# Если нужна информация о GPU, раскомментируйте после экспериментов с DDP:
# if torch.cuda.is_available():
#     for i in range(num_gpus):
#         print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
#         print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        
if num_gpus < 2:
    print("\n⚠️  WARNING: Для DataParallel и DDP рекомендуется минимум 2 GPU.")
    print("   Эксперименты 2 и 3 могут работать не оптимально или не запуститься.")


CUDA available: True
Number of GPUs available: 2


## Подготовка данных

Загрузим датасет AG News с оптимизированными настройками.


In [2]:
# Создаем DataModule с оптимизациями
dm = TextFeaturesDataModule(
    max_features=20000,
    batch_size=256,
    use_bigrams=True,
    cache_dir="./cache",
    num_workers=4,
    pin_memory=True,  # Оптимизация для GPU
)

dm.prepare_data()
dm.setup()

print(f"\nDataset info:")
print(f"  Input dim: {dm.input_dim}")
print(f"  Num classes: {dm.n_classes}")
print(f"  Train samples: {len(dm.train_dataset)}")
print(f"  Val samples: {len(dm.val_dataset)}")
print(f"  Batch size: {dm.batch_size}")


Loading from cache...
Dataset loaded: classes=4, features=20005, train=108460, val=19140

Dataset info:
  Input dim: 20005
  Num classes: 4
  Train samples: 108460
  Val samples: 19140
  Batch size: 256


## Эксперимент 1: Single GPU (Baseline)

Обучение на одной GPU - baseline для сравнения.


In [3]:
# # Создаем модель
# model_1 = DeepMLP(
#     input_dim=dm.input_dim,
#     output_dim=dm.n_classes,
#     hidden_dims=[2048, 1024, 512, 256, 128],
#     activation='silu',
#     dropout=0.3,
#     use_batch_norm=True,
# )

# lightning_model_1 = TextClassificationModule(
#     model=model_1,
#     lr=1e-3,
#     optimizer_type='adam',
#     weight_decay=1e-5,
#     num_classes=dm.n_classes,
# )

# # Профилировщик
# profiler_1 = SimpleProfiler(dirpath='./profiler', filename='exp1_single_gpu')

# # Trainer - Single GPU
# trainer_1 = pl.Trainer(
#     max_epochs=10,
#     accelerator='gpu' if torch.cuda.is_available() else 'cpu',
#     devices=1,  # Одна GPU
#     profiler=profiler_1,
#     enable_progress_bar=True,
# )

# print("\n" + "="*80)
# print("ЭКСПЕРИМЕНТ 1: Single GPU (Baseline)")
# print("="*80)
# print(f"Strategy: Single Device")
# print(f"Devices: 1 GPU")

# # Обучение с замером времени (без CUDA синхронизации для совместимости с DDP)
# start_time = time.time()

# trainer_1.fit(lightning_model_1, dm)

# training_time_1 = time.time() - start_time

# print(f"\n✓ Обучение завершено за {training_time_1:.2f} секунд ({training_time_1/60:.2f} минут)")

# # Очистка памяти (удаляем только Python объекты)
# del model_1, lightning_model_1, trainer_1
# # Не вызываем torch.cuda.empty_cache() чтобы избежать инициализации CUDA


In [4]:
# # Результаты профилировщика
# print("\n" + "="*80)
# print("РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 1")
# print("="*80 + "\n")
# print(profiler_1.summary())


## Эксперимент 2: DistributedDataParallel (DDP)

DDP - эффективная стратегия распределённого обучения:
- Каждая GPU имеет свою копию модели и оптимизатора
- Градиенты синхронизируются между GPU
- Более эффективное использование памяти и коммуникации
- Рекомендуется для продакшн обучения


In [5]:
if num_gpus >= 2:
    # Создаем модель
    model_3 = DeepMLP(
        input_dim=dm.input_dim,
        output_dim=dm.n_classes,
        hidden_dims=[2048, 1024, 512, 256, 128],
        activation='silu',
        dropout=0.3,
        use_batch_norm=True,
    )

    lightning_model_3 = TextClassificationModule(
        model=model_3,
        lr=1e-3,
        optimizer_type='adam',
        weight_decay=1e-5,
        num_classes=dm.n_classes,
    )

    # Профилировщик
    profiler_3 = SimpleProfiler(dirpath='./profiler', filename='exp3_ddp')

    # Trainer - DDP
    trainer_3 = pl.Trainer(
        max_epochs=10,
        accelerator='gpu',
        devices=num_gpus,  # Все доступные GPU
        strategy='ddp_notebook',  # DistributedDataParallel
        profiler=profiler_3,
        enable_progress_bar=True,
    )

    print("\n" + "="*80)
    print("ЭКСПЕРИМЕНТ 3: DistributedDataParallel (DDP)")
    print("="*80)
    print(f"Strategy: DDP")
    print(f"Devices: {num_gpus} GPUs")

    # Обучение с замером времени (без CUDA синхронизации)
    start_time = time.time()

    trainer_3.fit(lightning_model_3, dm)

    training_time_3 = time.time() - start_time

    print(f"\n✓ Обучение завершено за {training_time_3:.2f} секунд ({training_time_3/60:.2f} минут)")

    # Очистка памяти (удаляем только Python объекты)
    del model_3, lightning_model_3, trainer_3
    # Не вызываем torch.cuda.empty_cache() чтобы избежать инициализации CUDA
else:
    print("\n⚠️  Пропускаем эксперимент 3: недостаточно GPU для DDP")
    training_time_3 = None
    profiler_3 = None


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



ЭКСПЕРИМЕНТ 3: DistributedDataParallel (DDP)
Strategy: DDP
Devices: 2 GPUs


You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes


Loading from cache...Loading from cache...

Dataset loaded: classes=4, features=20005, train=108460, val=19140
Dataset loaded: classes=4, features=20005, train=108460, val=19140


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | DeepMLP            | 43.8 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | train_acc | MulticlassAccuracy | 0      | train
3 | val_acc   | MulticlassAccuracy | 0      | train
4 | val_f1    | MulticlassF1Score  | 0      | train
---------------------------------------------------------
43.8 M    Trainable params
0         Non-trainable params
43.8 M    Total params
175.072   Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.



✓ Обучение завершено за 106.98 секунд (1.78 минут)


In [6]:
if num_gpus >= 2 and profiler_3 is not None:
    # Результаты профилировщика
    print("\n" + "="*80)
    print("РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 3")
    print("="*80 + "\n")
    print(profiler_3.summary())



РЕЗУЛЬТАТЫ ПРОФИЛИРОВЩИКА - Эксперимент 3

Profiler Report


